In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import math
from queue import PriorityQueue
import copy
import heapq

# Function to create a tree graph interactively
def create_tree():
    n = int(input("Enter the number of nodes: "))
    G = nx.DiGraph()
    node_names = [input(f"Enter the name of node {i+1}: ") for i in range(n)]
    
    heuristics = {}
    for name in node_names:
        heuristic_value = int(input(f"Enter the heuristic value for node {name}: "))
        heuristics[name] = heuristic_value
    
    for i in range(n-1):
        parents_count = int(input(f"Enter the number of parents for {node_names[i+1]}: "))
        while parents_count > 0:
            parent = input(f"Enter the parent node {parents_count} for node {node_names[i+1]}: ")
            while True:
                try:
                    cost = int(input(f"Enter the cost from {parent} to node {node_names[i+1]}: "))
                    break  # Break the loop if a valid integer is provided
                except ValueError:
                    print("Invalid input. Please enter a valid integer for the cost.")
            G.add_edge(parent, node_names[i+1], cost=cost)
            parents_count -= 1
    
    return G, heuristics


# Function to visualize a tree graph
def visualize_tree(G):
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=1000, node_color="skyblue", font_size=10, font_weight="bold", font_color="black")
    edge_labels = nx.get_edge_attributes(G, 'cost')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    plt.show()

# Function to print the path and cost
def print_path(path, cost):
    print(" -> ".join(path))
    print(f"Cost: {cost}")

# Breadth-First Search (BFS) algorithm
def bfs(G, start_node, goal_node):
    shortest_path = nx.shortest_path(G, source=start_node, target=goal_node, method="bfs")
    cost = sum(G[shortest_path[i]][shortest_path[i + 1]]["cost"] for i in range(len(shortest_path) - 1))
    return shortest_path, cost

# Depth-First Search (DFS) algorithm
def dfs(G, start_node, goal_node):
    # Recursive DFS path finding function
    def dfs_path(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return path
        if start not in graph:
            return None
        shortest_path = None
        for neighbor in graph[start]:
            neighbor_name = neighbor
            if neighbor_name not in path:
                new_path = dfs_path(graph, neighbor_name, end, path)
                if new_path:
                    if shortest_path is None or len(new_path) < len(shortest_path):
                        shortest_path = new_path
        return shortest_path

    shortest_path = dfs_path(G, start_node, goal_node)
    cost = sum(G[shortest_path[i]][shortest_path[i + 1]]["cost"] for i in range(len(shortest_path) - 1)) if shortest_path else math.inf
    return shortest_path, cost

# Hill Climbing algorithm
def hill_climbing(G, start_node, goal_node, heuristics):
    current_node = start_node
    path = [current_node]
    total_cost = 0
    canceled_nodes = []

    while current_node != goal_node:
        neighbors = list(G.neighbors(current_node))
        min_cost = float('inf')
        next_node = None

        for neighbor in neighbors:
            edge_cost = G[current_node][neighbor].get('cost', 0)
            neighbor_heuristic = heuristics.get(neighbor, 0)
            if neighbor not in path and (edge_cost + neighbor_heuristic) < min_cost:
                min_cost = edge_cost + neighbor_heuristic
                next_node = neighbor

        if next_node is None:
            canceled_nodes.append(current_node)
            break

        path.append(next_node)
        current_node = next_node

    return path, canceled_nodes

# Beam Search algorithm
def beam_search(G, start_node, goal_node, beam_width):
    # Beam Search path finding function
    def beam_search_path(graph, start, end, beam_width):
        open_list = [(0, [start])]
        closed_set = set()

        while open_list:
            open_list.sort(key=lambda x: x[0])
            current_cost, path = open_list.pop(0)
            current_node = path[-1]

            if current_node == end:
                return path

            if current_node in closed_set:
                continue

            closed_set.add(current_node)

            neighbors = list(graph.neighbors(current_node))
            for neighbor in neighbors:
                if neighbor not in path:
                    neighbor_cost = graph[current_node][neighbor].get('cost', 0)
                    total_cost = current_cost + neighbor_cost
                    new_path = path + [neighbor]
                    open_list.append((total_cost, new_path))

            open_list.sort(key=lambda x: x[0])
            open_list = open_list[:beam_width]

        return []

    shortest_path = beam_search_path(G, start_node, goal_node, beam_width)
    cost = sum(G[shortest_path[i]][shortest_path[i + 1]]["cost"] for i in range(len(shortest_path) - 1)) if shortest_path else math.inf
    return shortest_path, cost

# Branch and Bound algorithm
def branch_and_bound(G, start_node, goal_node):
    # Recursive DFS path finding function
    def dfs_path(graph, start, end, path=[], visited_nodes=[]):
        path = path + [start]
        visited_nodes.append(start)
        if start == end:
            return path, visited_nodes
        if start not in graph:
            return None, visited_nodes
        shortest_path = None
        for neighbor in graph[start]:
            neighbor_name = neighbor
            if neighbor_name not in path:
                new_path, visited_nodes = dfs_path(graph, neighbor_name, end, path, visited_nodes)
                if new_path:
                    if shortest_path is None or len(new_path) < len(shortest_path):
                        shortest_path = new_path
        return shortest_path, visited_nodes

    shortest_path, visited_nodes = dfs_path(G, start_node, goal_node)
    cost = sum(G[shortest_path[i]][shortest_path[i + 1]]["cost"] for i in range(len(shortest_path) - 1)) if shortest_path else math.inf
    return shortest_path, visited_nodes, cost

# Branch and Bound Greedy algorithm
def branch_and_bound_greedy(G, start_node, goal_node):
    current_node = start_node
    path = [current_node]
    total_cost = 0
    final_path = [current_node]

    while current_node != goal_node:
        neighbors = list(G.neighbors(current_node))
        min_cost = float('inf')
        next_node = None

        for neighbor in neighbors:
            edge_cost = G[current_node][neighbor].get('cost', 0)
            if neighbor not in path:
                if edge_cost < min_cost:
                    min_cost = edge_cost
                    next_node = neighbor

        if next_node is None:
            break

        path.append(next_node)
        current_node = next_node
        total_cost += min_cost

    return path, total_cost

# A* Search algorithm
def a_star(G, start_node, goal_node, heuristics):
    open_set = PriorityQueue()
    open_set.put((0, start_node))
    came_from = {}
    g_score = {node: float('inf') for node in G.nodes}
    g_score[start_node] = 0
    f_score = {node: float('inf') for node in G.nodes}
    f_score[start_node] = heuristics[start_node]

    while not open_set.empty():
        _, current_node = open_set.get()

        if current_node == goal_node:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start_node)
            path.reverse()
            cost = g_score[goal_node]
            return path, cost

        for neighbor in G.neighbors(current_node):
            tentative_g_score = g_score[current_node] + G[current_node][neighbor].get('cost', 0)

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristics[neighbor]
                open_set.put((f_score[neighbor], neighbor))

    return [], math.inf

def main():
    G, heuristics = create_tree()
    visualize_tree(G)
    start_node = input("Enter the start node: ")
    goal_node = input("Enter the goal node: ")

    # Breadth-First Search (BFS)
    bfs_path, bfs_cost = bfs(G, start_node, goal_node)
    print("\nBreadth-First Search:")
    print_path(bfs_path, bfs_cost)

    # Depth-First Search (DFS)
    dfs_path, dfs_cost = dfs(G, start_node, goal_node)
    print("\nDepth-First Search:")
    print_path(dfs_path, dfs_cost)

    # Hill Climbing
    hill_climbing_path, canceled_nodes = hill_climbing(G, start_node, goal_node, heuristics)
    print("\nHill Climbing:")
    print(f"Path: {' -> '.join(hill_climbing_path)}")
    print(f"Canceled Nodes: {', '.join(canceled_nodes)}")

    # Beam Search
    beam_search_path, beam_search_cost = beam_search(G, start_node, goal_node, 2)
    print("\nBeam Search:")
    print_path(beam_search_path, beam_search_cost)

    # Branch and Bound
    branch_and_bound_path, visited_nodes, branch_and_bound_cost = branch_and_bound(G, start_node, goal_node)
    print("\nBranch and Bound:")
    print_path(branch_and_bound_path, branch_and_bound_cost)
    print(f"Visited Nodes: {', '.join(visited_nodes)}")

    # Branch and Bound Greedy
    branch_and_bound_greedy_path, branch_and_bound_greedy_cost = branch_and_bound_greedy(G, start_node, goal_node)
    print("\nBranch and Bound (Greedy):")
    print_path(branch_and_bound_greedy_path, branch_and_bound_greedy_cost)

    # A* Search
    a_star_path, a_star_cost = a_star(G, start_node, goal_node, heuristics)
    print("\nA* Search:")
    print_path(a_star_path, a_star_cost)

if __name__ == "__main__":
    main()
